In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.27.dev792. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd

df= pd.read_csv('dataset_v2.csv')

# Ajout d'une colonne vide
df['input'] = ''

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def alpaca_prompt(row):
    return f"""Ci-dessous se trouve une instruction qui décrit une tâche, associée à une entrée fournissant plus de contexte. Écrivez une réponse qui complète correctement la demande en français, Generer une histoire de jeu qui inclut des mini-challenges basé sur mecanique didactique et mini-scenario suivant:

### Instruction:
{row['INST']}

### Input:
{row['input']}

### Response:
{row['OUTPUT']}

{EOS_TOKEN}
"""

In [ ]:
# Apply the formatting function to each row
formatted_array = df.apply(alpaca_prompt, axis=1)

# Create a new DataFrame with one column: 'formatted_data'
formatted_df = pd.DataFrame({'text': formatted_array})

formatted_df

,text
0,Ci-dessous se trouve une instruction qui décri...
1,Ci-dessous se trouve une instruction qui décri...
2,Ci-dessous se trouve une instruction qui décri...
3,Ci-dessous se trouve une instruction qui décri...
4,Ci-dessous se trouve une instruction qui décri...
...,...
124,Ci-dessous se trouve une instruction qui décri...
125,Ci-dessous se trouve une instruction qui décri...
126,Ci-dessous se trouve une instruction qui décri...
127,Ci-dessous se trouve une instruction qui décri...


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
train_dict = DatasetDict({
    'train': Dataset.from_pandas(formatted_df)
})
train_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 129
    })
})

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dict['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/129 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
!pip install --pre -U xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 129 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.271800
2,1.223300
3,1.263100
4,1.114400
5,1.081600
6,1.069700
7,0.972300
8,0.872700
9,0.893700
10,0.811100


In [ ]:
alpaca_prompt = """Ci-dessous se trouve une instruction qui décrit une tâche, associée à une entrée fournissant plus de contexte. Écrivez une réponse qui complète correctement la demande en français, Generer une histoire de jeu qui inclut des mini-challenges basé sur mecanique didactique et mini-scenario suivant:

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
# Exemple cas deja entrainer
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Le joueur doit reconnaître et ordonner les lettres de l'alphabet niveau facile (MD) en complétant une série de mots manquants avec les lettres manquantes (MS).", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1800, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Ci-dessous se trouve une instruction qui décrit une tâche, associée à une entrée fournissant plus de contexte. Écrivez une réponse qui complète correctement la demande en français, Generer une histoire de jeu qui inclut des mini-challenges basé sur mecanique didactique et mini-scenario suivant:\n\n### Instruction:\nLe joueur doit reconnaître et ordonner les lettres de l\'alphabet niveau facile (MD) en complétant une série de mots manquants avec les lettres manquantes (MS).\n\n### Input:\n\n\n### Response:\nMC1 : Le joueur se retrouve dans une bibliothèque magique où les livres parlent par eux-mêmes. Pour avancer, il doit compléter une série de mots manquants en identifiant les lettres manquantes de l\'alphabet. Chaque mot correctement complété lui permet de déverrouiller une nouvelle section de la bibliothèque, révélant ainsi de précieux indices pour la suite de son aventure.\nMC2 : Le joueur se trouve face à un puzzle complexe composé de lettres dispersées dans un l

In [ ]:
# Save to 8bit Q8_0
#model.save_pretrained_gguf("model", tokenizer,)
model.push_to_hub_gguf("mohcinebd/gguf_model_llama3_dataset_v2", tokenizer, token = "")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.92 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:38<00:00,  3.09s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving mohcinebd/gguf_model_llama3_dataset_v2/pytorch_model-00001-of-00004.bin...
Unsloth: Saving mohcinebd/gguf_model_llama3_dataset_v2/pytorch_model-00002-of-00004.bin...
Unsloth: Saving mohcinebd/gguf_model_llama3_dataset_v2/pytorch_model-00003-of-00004.bin...
Unsloth: Saving mohcinebd/gguf_model_llama3_dataset_v2/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.
Unsloth: We must use f16 for non Llama and Mistral models.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q8_0 will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at mohcinebd/gguf_model_llama3_dataset_v2 into f16 GGUF format.
The output location will be ./mohcinebd/gguf_model_llama3_dataset_v2-unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: gguf_model_llama3_dataset_v2
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-

gguf_model_llama3_dataset_v2-unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/mohcinebd/gguf_model_llama3_dataset_v2
